In [12]:
# Imports
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from time import sleep

# Install the webdriver_manager pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install asyncio

In [38]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/matt/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


In [43]:
def start_kayak(city_from, city_to, date_start, date_end):
    """City codes - it's the IATA codes!
    Date format -  YYYY-MM-DD"""
    
    # Starting kayak chrome 
    kayak = ('https://www.kayak.co.uk/flights/' + city_from + '-' + city_to +
             '/' + date_start + date_end + 'sort=bestflight_a?sort=price_a')
    driver.get(kayak)
    sleep(5) # sleep to allow the website to load
    
    # If the privacy modal opens, click accept
    try:
        accept_button_xpath = '//button[contains(@id, "-accept")]'
        accept_button = driver.find_element(By.XPATH, accept_button_xpath)
        accept_button.click()
        sleep(1)
    except Exception as e:
        # try clicking accept all on the other flavour of privacy modal
        # try:
        print(e)
        
    actions = ActionChains(driver)
    
    # Click on filter to include flights only
#     try: 
#         plane_checkbox_xpath = '//div[contains(@id, "-transportation_plane-check-icon")]'
#         plane_only_xpath = '//button[contains(@id, "-transportation_plane-only")]'
#         plane_checkbox_div = driver.find_element(By.XPATH, plane_checkbox_xpath)
#         plane_only_button = driver.find_element(By.XPATH, plane_only_xpath)
        
#         actions.move_to_element(plane_checkbox_div)
#         actions.pause(3)
#         actions.click(plane_only_button)
#         actions.perform()
#         sleep(1)
#     except Exception as e:
#         print(e)
    
    

In [52]:
# functions to obtain each property of a dictionary which will then be the exported csv file

"""
{
origin: char(3)
destination: char(3)
time_start: int
time_end: int
date_start: date
date_end: date
price: int
href: string;
}
"""

# need to find a way to scrape a single flight, then iterate

def scrape_hrefs():
    """
    scrape the hrefs on the page
    """
#     href_xpath = '//span[contains(text(), "View Deal")]//parent::a[@class="booking-link "]' 
    href_xpath = '//a[contains(@aria-label, "View Deal:")]//span[@class="price-text"]//ancestor::a[contains(@aria-label, "View Deal:")]'
    anchors = driver.find_elements(By.XPATH, href_xpath)
    
    def get_href(anchor):
        return anchor.get_attribute("href")
    
    print(len(anchors))
    
    return list(map(get_href, anchors)) 

def scrape_prices(): 
    """
    scrape the prices on the page
    """
    
    price_xpath = '//a[contains(@aria-label, "View Deal:")]//span[@class="price-text"]'
    spans = driver.find_elements(By.XPATH, price_xpath)
    
    def get_price(span):
        return span.text;
    
    print(len(spans))
    
    return list(map(get_price, spans))

def scrape_start_times():
    """
    scrape the depart times
    """
    
    start_times_xpath = '//a[contains(@aria-label, "View Deal:")]//preceding::div[@class="section stacked-carriers with-date"]//following-sibling::div[@class="section times"]//descendant::span[@class="depart-time base-time"]'
    spans = driver.find_elements(By.XPATH, start_times_xpath)
    
    def get_time(span):
        return span.text
    
    print(len(spans))
    
    return list(map(get_time, spans))

def scrape_end_times():
    """
    scrape the depart times
    """
    
    start_times_xpath = '//a[contains(@aria-label, "View Deal:")]//preceding::div[@class="section stacked-carriers"]//following-sibling::div[@class="section times"]//descendant::span[@class="arrival-time base-time"]'
    spans = driver.find_elements(By.XPATH, start_times_xpath)
    
    def get_time(span):
        return span.text
    
    print(len(spans))
    
    return list(map(get_time, spans))

def scrape_date_start():
    """
    scrape date of departure and date of return
    """
    dates_xpath = '//a[contains(@aria-label, "View Deal:")]//preceding::div[@class="section stacked-carriers"]//div[@class="bottom"]//child::span'
    spans = driver.find_elements(By.XPATH, dates_xpath)
    
    def get_date(span):
        return span.text
    
    print(len(spans))
    
    return list(map(get_date, spans))

def scrape_airports():
    """
    """
    pass


In [44]:
start_kayak("LGW", "CDG", "2022-03-15", "2022-03-22")

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[contains(@id, "-accept")]"}
  (Session info: chrome=98.0.4758.102)



In [53]:
# print(scrape_hrefs())
# print(scrape_prices())
print(scrape_start_times())
print(scrape_end_times())
# print(scrape_date_start())

0
[]
5
['21:23', '16:05', '10:30', '08:35', '22:10']
